In [9]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
kaggle_api_token = {"username":"alaaelsham","key":"d6d73b3d388e1e7597e8326ce3fe567c"}
import json
with open('/Users/alaaelshamouty/.kaggle/kaggle.json', 'w') as file:
    json.dump(kaggle_api_token, file)

! chmod 600 ~/.kaggle/kaggle.json
! mkdir datasets_kaggle/

mkdir: /Users/alaaelshamouty/.kaggle: File exists


In [11]:
name = "kdd-cup-2014-predicting-excitement-at-donors-choose"
! kaggle competitions download -c {name}
!mkdir datasets_kaggle/{name}
!unzip {name}.zip -d datasets_kaggle/{name}

100%|███████████████████████████████████████▉| 926M/926M [02:36<00:00, 5.90MB/s]
100%|████████████████████████████████████████| 926M/926M [02:36<00:00, 6.21MB/s]
Archive:  kdd-cup-2014-predicting-excitement-at-donors-choose.zip
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/donations.csv.zip  
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/essays.csv.zip  
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/outcomes.csv.zip  
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/projects.csv.zip  
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/resources.csv.zip  
  inflating: datasets_kaggle/kdd-cup-2014-predicting-excitement-at-donors-choose/sampleSubmission.csv.zip  


In [14]:
import zipfile
import os

for file in os.listdir(f"datasets_kaggle/{name}/"):
    if ".zip" in file:
        print(f"unzipping {file}")
        with zipfile.ZipFile(f"datasets_kaggle/{name}/{file}", 'r') as zip_ref:
            zip_ref.extractall(f"datasets_kaggle/{name}/")


unzipping essays.csv.zip
unzipping sampleSubmission.csv.zip
unzipping projects.csv.zip
unzipping resources.csv.zip
unzipping outcomes.csv.zip
unzipping donations.csv.zip


In [1]:
import pandas as pd
import os
name = "kdd-cup-2014-predicting-excitement-at-donors-choose"
#datasets_stats = pd.DataFrame(columns=["table_name","nr_rows","nr_features"])
for file in os.listdir(f"datasets_kaggle/{name}/"):
    if ".csv" in file and not ".zip" in file and not "sample" in file:
        filename = file.split(".")
        print(f"sampling{filename[0]}")
        df = pd.read_csv(f"datasets_kaggle/{name}/{file}")
        df_new = df.sample(50, replace=False, random_state=1)
        df_new.to_csv(f"datasets_kaggle/{name}/{filename[0]}_sample_new.csv")


samplingdonations
samplingresources
samplingoutcomes
samplingessays
samplingprojects


In [1]:
import pandas as pd
name = "kdd-cup-2014-predicting-excitement-at-donors-choose"
# loading all dataframes
donations_df = pd.read_csv(f'datasets_kaggle/{name}/donations.csv')
essays_df = pd.read_csv(f'datasets_kaggle/{name}/essays_sample.csv')
outcomes_df = pd.read_csv(f'datasets_kaggle/{name}/outcomes.csv')
projects_df = pd.read_csv(f'datasets_kaggle/{name}/projects.csv')
resources_df = pd.read_csv(f'datasets_kaggle/{name}/resources_sample.csv')
sample_submission_df = pd.read_csv(f'datasets_kaggle/{name}/sampleSubmission.csv')

In [2]:
print(len(projects_df))

664098


# Applying Jimmy's Code

In [3]:
# Joining projects with outcomes
# Usefulness: This forms the base dataframe where we can add more features that will help in predicting the "is_exciting" label.
# Input samples: 'projectid' from projects_df: ['dd291f97dc6b32b7c9f052794efc4929', '673d26170aeaf573d6e4c7565a65915b', '8f0e3bc89dc5304f71f167201242fa5e']
#                'projectid' from outcomes_df: ['c6bc4035d501cc99ab78072d8544e3e4', 'cc7e0ca72280d0b08846ec7f15549406', 'a75a074419bcd15f3c22a8dcddd5f7cb']
merged_df = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [4]:
len(merged_df)

619326

In [6]:
#### Feature 1: `total_price_with_fulfillment`
#Calculate the total price of a project including the `fulfillment_labor_materials`.

# Feature: Total price of the project including fulfillment labor materials
# Usefulness: The total financial burden of a project, including hidden costs, may influence its chances of being considered "exciting."
# Input samples: 'total_price_excluding_optional_support': [2148.49, 227.37, 188.80], 'fulfillment_labor_materials': [30.0, 35.0, 35.0]
merged_df['total_price_with_fulfillment'] = merged_df['total_price_excluding_optional_support'] + merged_df['fulfillment_labor_materials']

#### Feature 2: `students_per_dollar`
#Calculate the number of students reached per dollar spent on the project.

# Feature: Number of students reached per dollar
# Usefulness: Projects that impact more students per dollar may be more "exciting" due to their cost-effectiveness.
# Input samples: 'students_reached': [125.0, 62.0, 80.0], 'total_price_with_fulfillment': [2178.49, 262.37, 223.80]
merged_df['students_per_dollar'] = merged_df['students_reached'] / merged_df['total_price_with_fulfillment']

#### Drop Columns
#We will drop the columns that are already transformed or do not add much information to the predictive model.

# Explanation: Dropping columns that have been transformed or are not directly useful for predicting 'is_exciting'
#merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'total_price_excluding_optional_support', 'fulfillment_labor_materials'], inplace=True)
merged_df.drop(columns=['total_price_excluding_optional_support', 'fulfillment_labor_materials'], inplace=True)

In [8]:
merged_df['total_price_with_fulfillment'].head()

0    474.36
1    263.24
2    315.09
3    262.94
4    543.41
Name: total_price_with_fulfillment, dtype: float64

In [13]:
# Joining merged_df with donations_df on projectid
# Usefulness: This allows us to add features based on the donations each project received, which could be a strong indicator of whether a project is "exciting."
# Input samples: 'projectid' from merged_df: ['195d03cc23a38b2990dff4b81e774f28', '3c884a0ee29b7dea836b887d60d79c08', 'ce4c4ef12a619490c211f9534797796b']
#                'projectid' from donations_df: ['54622a1620e0047229ab1a466f2092dd', 'be8d90d9c4c8f4cbe8e75b39a835f46a', 'b5a6a8c2df3137d3be7f1b0d03f7ffed']
merged_with_donations_df = pd.merge(merged_df, donations_df, on='projectid', how='left')

#### Feature 3: `average_donation_amount`
#Calculate the average donation amount received by each project.

# Feature: Average donation amount for each project
# Usefulness: A higher average donation amount could indicate that donors find the project more compelling or "exciting."
# Input samples: 'donation_total': [17.65, 25.00, 100.00]
merged_with_donations_df['average_donation_amount'] = merged_with_donations_df.groupby('projectid')['donation_total'].transform('mean')

#### Feature 4: `donor_teacher_ratio`
# Calculate the ratio of teacher to non-teacher donors for each project.

# Feature: Ratio of teacher donors to non-teacher donors for each project
# Usefulness: A higher ratio of teacher donors could indicate that the project is more "exciting" to educators, which may be a good sign.
# Input samples: 'is_teacher_acct': ['t', 'f', 'f']
merged_with_donations_df['is_teacher'] = merged_with_donations_df['is_teacher_acct'].apply(lambda x: 1 if x == 't' else 0)
merged_with_donations_df['teacher_donor_ratio'] = merged_with_donations_df.groupby('projectid')['is_teacher'].transform('mean')

#### Drop Columns
# We'll drop columns that have been used for aggregations or transformations and won't be directly useful for the model.

# Explanation: Dropping columns that have been used for aggregation or transformation
#merged_with_donations_df.drop(columns=['donation_total', 'is_teacher_acct', 'is_teacher'], inplace=True)

In [15]:
merged_with_donations_df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,donation_message,average_donation_amount,is_teacher,teacher_donor_ratio
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,t,f,f,f,f,f,NaN,87.13,0,0.0
1,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,t,f,f,f,f,f,HSBC is proud to support this important educat...,87.13,0,0.0
2,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,f,f,t,f,f,f,I live in Bartlett and have 2 kids. I hope thi...,87.13,0,0.0
3,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,t,f,f,f,f,f,All the best!,87.13,0,0.0
4,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,f,t,f,f,f,f,I gave because I wanted to support kids in the...,87.13,0,0.0


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assume merged_with_donations_sampled_df is the final DataFrame after feature engineering
# Convert 'is_exciting' to numerical labels
label_encoder = LabelEncoder()
merged_with_donations_df['is_exciting'] = label_encoder.fit_transform(merged_with_donations_df['is_exciting'])

#exclude projects that are in the sample submission file as they are the test set
# Create a set of project IDs that are in the sample_submission_df
submission_project_ids = set(sample_submission_df['projectid'])

# Filter projects_df to only include projects that are in the sample_submission_df
train_df = merged_with_donations_df[~merged_with_donations_df['projectid'].isin(submission_project_ids)]

# Select features and target variable
features = ['total_price_with_fulfillment', 'students_per_dollar', 'average_donation_amount', 'teacher_donor_ratio']
X = train_df[features]
y = train_df['is_exciting']

# Drop rows with missing values in feature columns
X_dropped = X.dropna()
# Drop corresponding rows in target variable
y_dropped = y.loc[X_dropped.index]


In [11]:
X_dropped.isna().sum(axis=1)
len(X_dropped)

760780

In [12]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9809669023896527


In [13]:
X_dropped.columns

Index(['total_price_with_fulfillment', 'students_per_dollar',
       'average_donation_amount', 'teacher_donor_ratio'],
      dtype='object')

In [51]:
y_pred.sum()

251

## For Kaggle submission

In [14]:
filtered_projects_df = projects_df[projects_df['projectid'].isin(submission_project_ids)]

In [19]:
filtered_projects_df

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,total_price_with_fulfillment,students_per_dollar
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662.0,NaN,...,Books,highest poverty,Grades 6-8,653.89,32.0,f,f,2014-05-12,585.81,0.054625
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243.0,urban,...,Books,highest poverty,Grades PreK-2,348.79,22.0,f,f,2014-05-12,326.47,0.067388
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017.0,rural,...,Technology,high poverty,Grades PreK-2,506.93,17.0,f,f,2014-05-11,460.89,0.036885
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226.0,urban,...,Books,highest poverty,Grades 3-5,677.73,12.0,f,f,2014-05-11,606.07,0.019800
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006.0,urban,...,Other,highest poverty,Grades PreK-2,480.47,24.0,f,f,2014-05-11,438.40,0.054745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44767,f1e36a9d4d356db460977ea131449839,4f3ac5dc5df47d942efbcaecfb5534b4,cd957305d272ad4dbcca706b27774ec3,1.921630e+11,42.673767,-91.902444,Oelwein,IA,50662.0,NaN,...,Books,highest poverty,Grades PreK-2,232.81,21.0,f,f,2014-01-01,227.89,0.092150
44768,a63cdaaac3835e0582befd9c77f6a3f5,36410d7f061c234a1f86c4cee56d8b44,e540f61e9d98935f163a049abeaaab26,1.201440e+11,28.499796,-81.296813,Orlando,FL,32822.0,suburban,...,Books,highest poverty,Grades 3-5,289.19,30.0,f,f,2014-01-01,275.81,0.108771
44769,ac73fd0822adb3104c110e4e3fe3498d,feae8b241b3a114bed92504de9b97402,5a0db5dd6211cb9660cbda3b97fa76a1,6.227100e+10,34.099316,-118.352051,Los Angeles,CA,90046.0,urban,...,Supplies,high poverty,Grades PreK-2,532.41,20.0,f,f,2014-01-01,482.55,0.041446
44770,3a09c88cbfe67aaedc451d813d157aed,f94787826148f11768ac9d63990fd552,707bbed4ee89431ede35cf0c4f4644f8,NaN,38.695172,-90.245537,Saint Louis,MO,63120.0,urban,...,Supplies,highest poverty,Grades PreK-2,195.85,20.0,f,f,2014-01-01,196.47,0.101797


In [18]:
#### Feature 1: `total_price_with_fulfillment`
#Calculate the total price of a project including the `fulfillment_labor_materials`.

# Feature: Total price of the project including fulfillment labor materials
# Usefulness: The total financial burden of a project, including hidden costs, may influence its chances of being considered "exciting."
# Input samples: 'total_price_excluding_optional_support': [2148.49, 227.37, 188.80], 'fulfillment_labor_materials': [30.0, 35.0, 35.0]
filtered_projects_df['total_price_with_fulfillment'] = filtered_projects_df['total_price_excluding_optional_support'] + filtered_projects_df['fulfillment_labor_materials']

#### Feature 2: `students_per_dollar`
#Calculate the number of students reached per dollar spent on the project.

# Feature: Number of students reached per dollar
# Usefulness: Projects that impact more students per dollar may be more "exciting" due to their cost-effectiveness.
# Input samples: 'students_reached': [125.0, 62.0, 80.0], 'total_price_with_fulfillment': [2178.49, 262.37, 223.80]
filtered_projects_df['students_per_dollar'] = filtered_projects_df['students_reached'] / filtered_projects_df['total_price_with_fulfillment']

#### Drop Columns
#We will drop the columns that are already transformed or do not add much information to the predictive model.

# Explanation: Dropping columns that have been transformed or are not directly useful for predicting 'is_exciting'
#merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'total_price_excluding_optional_support', 'fulfillment_labor_materials'], inplace=True)
filtered_projects_df.drop(columns=['total_price_excluding_optional_support', 'fulfillment_labor_materials'], inplace=True)

KeyError: 'total_price_excluding_optional_support'

In [30]:
filtered_projects_df.projectid

0        316ed8fb3b81402ff6ac8f721bb31192
1        90de744e368a7e4883223ca49318ae30
2        32943bb1063267de6ed19fc0ceb4b9a7
3        bb18f409abda2f264d5acda8cab577a9
4        24761b686e18e5eace634607acbcc19f
                       ...               
44767    f1e36a9d4d356db460977ea131449839
44768    a63cdaaac3835e0582befd9c77f6a3f5
44769    ac73fd0822adb3104c110e4e3fe3498d
44770    3a09c88cbfe67aaedc451d813d157aed
44771    16bf6f4c3c37da1cb2e92ee8ce834510
Name: projectid, Length: 44772, dtype: object

In [26]:
donations_full_df = pd.read_csv(f'datasets_kaggle/{name}/donations.csv')

In [52]:
donations_full_df.projectid.isin(filtered_projects_df)

0          False
1          False
2          False
3          False
4          False
           ...  
3097984    False
3097985    False
3097986    False
3097987    False
3097988    False
Name: projectid, Length: 3097989, dtype: bool

In [51]:
merged_with_donations_df = pd.merge(filtered_projects_df, donations_df, on='projectid', how='left')
merged_with_donations_df[merged_with_donations_df["donation_total"].isna()]

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,dollar_amount,donation_included_optional_support,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,donation_message
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017.0,rural,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44767,f1e36a9d4d356db460977ea131449839,4f3ac5dc5df47d942efbcaecfb5534b4,cd957305d272ad4dbcca706b27774ec3,1.921630e+11,42.673767,-91.902444,Oelwein,IA,50662.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44768,a63cdaaac3835e0582befd9c77f6a3f5,36410d7f061c234a1f86c4cee56d8b44,e540f61e9d98935f163a049abeaaab26,1.201440e+11,28.499796,-81.296813,Orlando,FL,32822.0,suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44769,ac73fd0822adb3104c110e4e3fe3498d,feae8b241b3a114bed92504de9b97402,5a0db5dd6211cb9660cbda3b97fa76a1,6.227100e+10,34.099316,-118.352051,Los Angeles,CA,90046.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44770,3a09c88cbfe67aaedc451d813d157aed,f94787826148f11768ac9d63990fd552,707bbed4ee89431ede35cf0c4f4644f8,NaN,38.695172,-90.245537,Saint Louis,MO,63120.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
merged_with_donations_df['average_donation_amount'] = merged_with_donations_df.groupby('projectid')['donation_total'].transform('mean')

In [49]:
merged_with_donations_df.groupby('projectid')['donation_total'].head()

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
44767   NaN
44768   NaN
44769   NaN
44770   NaN
44771   NaN
Name: donation_total, Length: 44772, dtype: float64

In [33]:
 #Joining merged_df with donations_df on projectid
# Usefulness: This allows us to add features based on the donations each project received, which could be a strong indicator of whether a project is "exciting."
# Input samples: 'projectid' from merged_df: ['195d03cc23a38b2990dff4b81e774f28', '3c884a0ee29b7dea836b887d60d79c08', 'ce4c4ef12a619490c211f9534797796b']
#                'projectid' from donations_df: ['54622a1620e0047229ab1a466f2092dd', 'be8d90d9c4c8f4cbe8e75b39a835f46a', 'b5a6a8c2df3137d3be7f1b0d03f7ffed']
merged_with_donations_df = pd.merge(filtered_projects_df, donations_df, on='projectid', how='left')

#### Feature 3: `average_donation_amount`
#Calculate the average donation amount received by each project.

# Feature: Average donation amount for each project
# Usefulness: A higher average donation amount could indicate that donors find the project more compelling or "exciting."
# Input samples: 'donation_total': [17.65, 25.00, 100.00]
merged_with_donations_df['average_donation_amount'] = merged_with_donations_df.groupby('projectid')['donation_total'].transform('mean')

#### Feature 4: `donor_teacher_ratio`
# Calculate the ratio of teacher to non-teacher donors for each project.

# Feature: Ratio of teacher donors to non-teacher donors for each project
# Usefulness: A higher ratio of teacher donors could indicate that the project is more "exciting" to educators, which may be a good sign.
# Input samples: 'is_teacher_acct': ['t', 'f', 'f']
merged_with_donations_df['is_teacher'] = merged_with_donations_df['is_teacher_acct'].apply(lambda x: 1 if x == 't' else 0)
merged_with_donations_df['teacher_donor_ratio'] = merged_with_donations_df.groupby('projectid')['is_teacher'].transform('mean')

#### Drop Columns
# We'll drop columns that have been used for aggregations or transformations and won't be directly useful for the model.

# Explanation: Dropping columns that have been used for aggregation or transformation
merged_with_donations_df.drop(columns=['donation_total', 'is_teacher_acct', 'is_teacher'], inplace=True)

In [36]:
merged_with_donations_df

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,donation_message,average_donation_amount,teacher_donor_ratio
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017.0,rural,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44767,f1e36a9d4d356db460977ea131449839,4f3ac5dc5df47d942efbcaecfb5534b4,cd957305d272ad4dbcca706b27774ec3,1.921630e+11,42.673767,-91.902444,Oelwein,IA,50662.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
44768,a63cdaaac3835e0582befd9c77f6a3f5,36410d7f061c234a1f86c4cee56d8b44,e540f61e9d98935f163a049abeaaab26,1.201440e+11,28.499796,-81.296813,Orlando,FL,32822.0,suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
44769,ac73fd0822adb3104c110e4e3fe3498d,feae8b241b3a114bed92504de9b97402,5a0db5dd6211cb9660cbda3b97fa76a1,6.227100e+10,34.099316,-118.352051,Los Angeles,CA,90046.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
44770,3a09c88cbfe67aaedc451d813d157aed,f94787826148f11768ac9d63990fd552,707bbed4ee89431ede35cf0c4f4644f8,NaN,38.695172,-90.245537,Saint Louis,MO,63120.0,urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [37]:
# Select features and target variable
features = ['total_price_with_fulfillment', 'students_per_dollar', 'average_donation_amount', 'teacher_donor_ratio']
X_test = merged_with_donations_df[features]
X_test
# Make predictions
#y_pred = clf.predict(X_test)

,total_price_with_fulfillment,students_per_dollar,average_donation_amount,teacher_donor_ratio
0,585.81,0.054625,NaN,0.0
1,326.47,0.067388,NaN,0.0
2,460.89,0.036885,NaN,0.0
3,606.07,0.019800,NaN,0.0
4,438.40,0.054745,NaN,0.0
...,...,...,...,...
44767,227.89,0.092150,NaN,0.0
44768,275.81,0.108771,NaN,0.0
44769,482.55,0.041446,NaN,0.0
44770,196.47,0.101797,NaN,0.0


In [22]:
X_test

,total_price_with_fulfillment,students_per_dollar,average_donation_amount,teacher_donor_ratio
0,585.81,0.054625,NaN,0.0
1,326.47,0.067388,NaN,0.0
2,460.89,0.036885,NaN,0.0
3,606.07,0.019800,NaN,0.0
4,438.40,0.054745,NaN,0.0
...,...,...,...,...
44767,227.89,0.092150,NaN,0.0
44768,275.81,0.108771,NaN,0.0
44769,482.55,0.041446,NaN,0.0
44770,196.47,0.101797,NaN,0.0


# Without FE just main table with feature selection

In [35]:
merged_df_new = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming merged_df is your main DataFrame after merging projects_df and outcomes_df
# Convert 'is_exciting' to numerical labels for machine learning
label_encoder = LabelEncoder()
merged_df_new['is_exciting'] = label_encoder.fit_transform(merged_df_new['is_exciting'].fillna(''))

# Select some existing features (before feature engineering) for demonstration
# We'll consider 'students_reached' and 'total_price_excluding_optional_support' for this example
features = ['students_reached', 'total_price_excluding_optional_support']

# Drop rows with missing values in feature columns
X_dropped = merged_df_new[features].dropna()

# Drop corresponding rows in target variable
y_dropped = merged_df_new['is_exciting'].loc[X_dropped.index]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.90465746355309


# Without FE just main table without feature selection

In [42]:
merged_df_new_new = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [44]:
# Assuming merged_df is your main DataFrame after merging projects_df and outcomes_df
# Convert 'is_exciting' to numerical labels for machine learning
label_encoder = LabelEncoder()
merged_df_new_new['is_exciting'] = label_encoder.fit_transform(merged_df_new_new['is_exciting'].fillna(''))

# Drop rows with missing values across all columns
clean_df = merged_df_new_new.dropna()

# Filter only numerical features
numerical_features = clean_df.select_dtypes(include=['float64', 'int64']).columns

# Separate features and target variable
X_dropped = clean_df[numerical_features]  # Keep only numerical features
X_dropped = X_dropped.drop(columns=['is_exciting'])
y_dropped = clean_df['is_exciting']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9824506424318395


In [45]:
numerical_features

Index(['Unnamed: 0_x', 'school_ncesid', 'school_latitude', 'school_longitude',
       'school_zip', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'Unnamed: 0_y', 'is_exciting', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count'],
      dtype='object')